# Yeast cells detection sample notebook

In [1]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detecron2
import numpy as np

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.output as output
import yeastcells.visualize as visualize

%matplotlib notebook

In [2]:
cd ~

/home/p301081


### Load data

In [3]:
path = '../../slow-data/yeastcells/data/DATA_CIT/Test_Images'
filenames = data.load_data(path, ff = '.tif')
print(filenames)

['../../slow-data/yeastcells/data/DATA_CIT/Test_Images/Movie_test1.tif']


In [4]:
fn = filenames[0]
image = data.read_image(fn, single_im = False, shape=2)
np.shape(image) #(frames, length, width, channels)

(300, 512, 512, 3)

In [5]:
image = image[39:]
len(image)

261

### Load and run model on data for segmentation

In [6]:
#set path to model_final.pth
model_path = '/var/tensorflow-logs/p253591/yeast-cell-detection-run-4'

#load model
predictor = model.load_model(model_path, seg_thresh=0.94, device='cuda:0') #set GPU if available otherwise use 'cpu'

In [7]:
output = [
  {'instances': predictor(frame)['instances'].to('cpu')}
  for frame in image
]

/home/p301081/.virtualenvs/yeastcells/lib/python3.7/site-packages/detectron2/layers/wrappers.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


### Run clustering algorithm for tracking

In [8]:
labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=5, eps=0.6, progress=False) #for time-series
#labels, coordinates = get_instance_numbers(output) #for single image

/home/p301081/.virtualenvs/yeastcells/lib/python3.7/site-packages/sklearn/neighbors/_base.py:168: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  EfficiencyWarning)


In [ ]:
#plot paths
ax = visualize.plot_paths(
  labels,
  coordinates[:, [2,1,0]],
  style={'linewidth': 1},
  xlim=(0, 512),
  ylim=(0, 512),
  title='',
)

### Extract features and visualize tracking and segmentation

In [10]:
s, t = features.get_seg_track(labels, output, frame = None) # total

The number of segmentations is 16125
The number of tracked cells is 431


In [11]:
s, t = features.get_seg_track(labels, output, frame = 40) # by frame

The number of segmentations in frame 40 is 21
The number of tracked cells in frame 40 is 20


In [14]:
features.track_len(labels, label=0) #select a label to extract the length of its track

255

In [15]:
contours = features.extract_contours(output) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=[255,0,255], framenum=True, labelnum=True)
movie = visualize.show_animation(scene, title='', delay=100) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=True, labelnum=True)
cell_scene, label = visualize.select_cell(scene, coordinates, labels, w=40, l=0) # choose image window (w) and label (l)
movie = visualize.show_animation(cell_scene, title='', delay=100)

In [ ]:
polygons_clust =  features.polygons_per_cluster(labels, contours, output)
ax = visualize.plot_area_profiles(polygons_clust, ti=3, label=[0], ax=None, title=None)

In [ ]:
_, _, pred_df = data.get_pred(output, labels, coordinates)
polygons_inst =  features.polygons_per_instance(labels, contours, output)
pred_features = features.get_features_df(polygons_inst, labels, pred_df)

In [ ]:
masks = features.get_masks(output)
visualize.plot_polygon_mask(
    masks, labels, output, images, polygons_clust, 
    label=0, frame=0, ax=None, title=None
)

### Output

In [ ]:
pred_df